<H2>Data Science Capstone Week 3 Assignment - PART 1</H2>
Intent:  Scrape a Wikipedia page and extract a pandas dataframe containing Toronto Postalcodes, Borough, and Neighborhood

<H3>Import Pandas Library</H3> 
...to provide the Dataframe container as well as the scraper "read_html" command

In [242]:
import pandas as pd
import numpy as np

<H3>Import Urllib Library</H3> 
...to read the HTML code into a string variable "jenhtml" ... this will facilitate the execution of pandas.read_html the  scaping command

In [243]:
import urllib.request
import urllib.parse
import urllib.error

!conda install -c conda-forge lxml --yes 
import lxml 

print("packages imported")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

packages imported


<H3>Open and Read Wikipedia Page into a String Variable</H3> 
...to read the HTML code into a string variable "jenhtml" ... this will facilitate the execution of pandas.read_html the  scraping command

In [244]:
jenurl = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
print(jenurl)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


In [245]:
jenhtmlStr = urllib.request.urlopen(jenurl).read()

In [246]:
jendf = pd.read_html(jenhtmlStr)[0]
jendf.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [247]:
jendf.shape

(288, 3)

<H3>Drop Rows with Missing Boroughs</H3> 

In [248]:
jendf = jendf[jendf.Borough != "Not assigned"]
jendf.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


<H1>Combine Rows of Postcode-sharing Neighborhoods</H1> 
<LI> determine which postcodes have more than one neighborhood
<LI> for each multiply-used postal code:<p>
 <p> Replace all multiply-used rows of the same postal code with just one row. <p> The new value for the "Neighbourhood" field will be a list of the former neighborhoods that appeared in the individual rows corresponding to the shared zipcode.

<H4> Find the multiply-used Postcodes by performing a df.value_counts() </H4>

In [249]:
mycounts = jendf["Postcode"].value_counts()
mycounts.head()

M8Y    8
M9V    8
M5V    7
M9B    5
M8Z    5
Name: Postcode, dtype: int64

<B>Form a LIST of Postcodes that have more than one neighborhood

In [250]:
postcodeList = mycounts.index.values.tolist()
pcodeList = []
for pcode in postcodeList:
    if mycounts.loc[pcode] > 1:
        pcodeList.append(str(pcode))
        
print(len(pcodeList))
neighList = []
boroList = []

57


<H2> Update the Dataframe by deleting duplications and augmenting 'Neighborhood' if necessary </H2>
<I>The algorithm employed is:</I>

For each postal code "pc" in pcodeList (created above):

1. Select and extract all rows in main dataframe "jendf"  with postalcode "pc" and place into another
   temporary dataframe called 'tempDF'
2. Form a new neighborhood String by extracting all individual neighborhoods
   in tempDF, putting them in a list, and invoking "join" with comma as delimiter
3. Save the new neighborhood string 'nStr' in a list; save its borough in a list called "boroList"

After the loop, save all collected information in a dictionary:  "nndict"
 
-----------------

In [251]:
#initialize lists
boroList = []
neighList = []
nList = list()

# Traverse pcodeList to create the new 'Neighbourhood' strings
for pc in pcodeList:
    nList = []
    # 1. select and extract all rows in main dataframe "jendf" with
    #    Postalcode "pc" and place into another temporary dataframe 'tempDF'
    tempdfSeries = jendf.Postcode == pc
    tempDF = jendf[tempdfSeries]
    # ----------------------------------------  
    # 2a.  Form a new neighborhood String by extracting all individual neighborhoods
    #     in tempDF and putting them in a list 
    nList = tempDF['Neighbourhood'].values.tolist()
    # ----------------------------------------
    # 2b. invoke python function 'join()' with comma as delimiter
    jenDelimiter = ', '
    nStr = jenDelimiter.join(nList)
    # ----------------------------------------
    #3.   Save the new neighborhood string 'nStr' in a list; save its borough in a list
    neighList.append(nStr)
    jenBoro = str(tempDF.iloc[0,1])
    boroList.append(jenBoro)
## end FOR LOOP

# Store all the collected information in a "New Neighbourhood" dictionary
nnDict = dict()

nnDict['Postcode'] = pcodeList
nnDict['Borough'] = boroList
nnDict['Neighbourhood'] = neighList
 

------
<H3> Remove Redundancy in Dataframe by deleting multiple rows and finally replacing 'Neighborhood' if necessary </H3>

<I>The algorithm employed is:</I>

A. Delete all postalcode duplicates from our original DF called "jendf" <p>
B. Create a new pandas dataframe "newNeighDF" to concatenate with our original DF "jendf" <p>
C. Merge the new dataframe "newNeighDF" into original "jendf" by using the pandas "concat" command

-----------------

<H4> A.   Delete all postalcode duplicates  </H4>

In [252]:
# A.   Delete all postalcode duplicates
jendf.drop_duplicates('Postcode', keep=False, inplace=True)

---------------
<H4> B. Create a new pandas dataframe "newNeighDF" to concatenate with our original DF "jendf" </H4>

In [254]:
newNeighDF =pd.DataFrame(data=nnDict)
newNeighDF.head()

,Postcode,Borough,Neighbourhood
0,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
1,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
2,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."
3,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ..."
4,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."


<H4>C. Merge the new dataframe "newNeighDF" into original "jendf" by using the pandas "concat" command
 </H4>

In [256]:
jenFrames = [jendf, newNeighDF]
postalDF = pd.concat(jenFrames)
postalDF.reset_index(drop=True,inplace=True)
postalDF.head(20)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M7A,Queen's Park,Not assigned
3,M9A,Etobicoke,Islington Avenue
4,M3B,North York,Don Mills North
5,M6B,North York,Glencairn
6,M4C,East York,Woodbine Heights
7,M5C,Downtown Toronto,St. James Town
8,M6C,York,Humewood-Cedarvale
9,M4E,East Toronto,The Beaches


--------------
<H1> Replace "Not assigned" Neighbourhood values </H1>
Employ the following algorithm:<p>
    <LI> (1) Extract all rows in dataframe <b>postalDF </b> with <b>Neighbourhood = 'Not assigned'</b> into a new dataframe <b>naNeighDF</b>
    <LI> (2) In dataframe <b>naNeighDF</b> assign "Neighbourhood" column to be equal to "Borough" column
    <LI> (3) Delete all rows from original dataframe <b>postalDF </b> with <b>Neighbourhood = 'Not assigned'</b>
    <LI> (4) Concatenate the two dataframes <b>postalDF </b> and <b>naNeighDF</b> into regionsTorontoDF        

In [257]:
# (1) Extract all rows in dataframe 'newNeighDF' with Neighbourhood = 'Not assigned' into a new dataframe naNeighDF
naNeighDF = postalDF[postalDF.Neighbourhood=='Not assigned']
naNeighDF

,Postcode,Borough,Neighbourhood
2,M7A,Queen's Park,Not assigned


In [258]:
#(2) In dataframe "naNeighDF" assign "Neighbourhood" column to be equal to "Borough" column
naNeighDF['Neighbourhood']=naNeighDF['Borough']
naNeighDF


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Postcode,Borough,Neighbourhood
2,M7A,Queen's Park,Queen's Park


In [259]:
naNeighDF.head(10)

,Postcode,Borough,Neighbourhood
2,M7A,Queen's Park,Queen's Park


In [260]:
# (3) Delete all rows from original dataframe "postalDF" with "Neighbourhood = 'Not assigned'"
postalDF = postalDF[postalDF.Neighbourhood!='Not assigned']

In [261]:
#(4) Concatenate the two dataframes <b>postalDF </b> and <b>naNeighDF</b> into regionsTorontoDF 
myframes = [postalDF, naNeighDF]
regionsTorontoDF = pd.concat(myframes)
regionsTorontoDF.shape

(103, 3)

In [262]:
regionsTorontoDF.reset_index(drop=True,inplace=True)
regionsTorontoDF.head(14)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M9A,Etobicoke,Islington Avenue
3,M3B,North York,Don Mills North
4,M6B,North York,Glencairn
5,M4C,East York,Woodbine Heights
6,M5C,Downtown Toronto,St. James Town
7,M6C,York,Humewood-Cedarvale
8,M4E,East Toronto,The Beaches
9,M5E,Downtown Toronto,Berczy Park


In [263]:
regionsTorontoDF.shape

(103, 3)